In [ ]:
# Import Functions
import pandas as pd
import os
import time
import csv
import json
import fnmatch
import csv
import sqlalchemy as sql
import numpy as np
dbHost='mxnunx41'
dbPort='1521'
dbUser='lims'
dbPassword='Mumwit63'
dbName='hwi1'
dburl='oracle://'
json_path = 'WaterActivity.json'

# Default Parameters setting
with open(json_path, 'r') as f:
    config = json.load(f)
    lab = config['lab']
    analysis = config['analysis']
    inst = config['inst']
    mode = config['mode']

colsOrder=['Sample.Sample_Number','Test.Test_Number','Test.Replicate_Count','Result.Result_Number','Result.Entry','Result.Entered_By']

# get Data from DB
def getDataFromDB(sampleNumbersList):
    dburl = 'oracle://'+dbUser+':'+dbPassword+'@'+dbHost+':'+dbPort+'/'+dbName
    engine=sql.create_engine(dburl)
    resultdf=pd.DataFrame()
    sampleNumbrStr=(', '.join("'" +str(item)+ "'" for item in sampleNumbersList))
    aly = ','.join("'"+x+"'" for x in analysis)
   # print(aly)
    # Query Needs to be changed according to the requirement
    queryText = 'SELECT r.result_number,r.name AS "Name" ,t.test_number,t.sample_number,t.replicate_count AS "Test_Replicate",r.replicate_count AS "Result_Replicate",t.analysis ,t.status,r.status FROM test t,RESULT r WHERE t.test_number = r.test_number AND t.STATUS NOT IN (\'X\',\'R\')  AND r.STATUS NOT IN (\'X\',\'R\')  AND t.analysis IN ('+aly+') AND t.sample_number= ('+sampleNumbrStr+')';
    print(queryText)
    conn = engine.connect()
    resultdf = pd.read_sql(queryText, engine)
    conn.close()
    return resultdf

#write the file
def writeFile(dframe,lab,inst,archDestination,importDest,datapath,fname):
    if os.path.exists(importDest):
        pass
    else:
        os.makedirs(importDest)
    timestr = time.strftime("%Y%m%d%H%M%S")
    for index, row in dframe.iterrows():
        _dframe = pd.DataFrame([row])
        print(row)
        FileName = importDest + lab+''+inst+'' + timestr +'_'+index+ '.csv'
        _dframe.to_csv(FileName, index=False, encoding="utf-8",quotechar='"',quoting=csv.QUOTE_NONNUMERIC) # write  out the file to be imported
    destination = archDestination + fname[:-4] + '.bak' # Backup the processed file
    #print(archDestination)
    #print(datapath)
    os.rename(datapath, destination)
    print("Processed File: " + fname[:-4])
    time.sleep(1)
#     FileName = importDest + lab+'_'+inst+'_' + timestr + '.csv'
#     dframe.to_csv(FileName, index=False, encoding="utf-8",quotechar='"',quoting=csv.QUOTE_NONNUMERIC) # write  out the file to be imported
#     destination = archDestination + fname[:-4] + '.bak' # Backup the processed file
#     os.rename(datapath, destination)
#     print("Processed File: " + fname[:-4])
#     time.sleep(1)


for eachlab in lab:
    print('Lab:'+eachlab)
    # Path needs to be changed according to the requirement
    root = '//mxns.loc//shares//NA-Instruments//Prod//'
    #path= root+eachlab+'//Import//'+inst+'//Datafolder//'
    path='C:/Users/in_kummay/Downloads/Downloads-local/WaterActivityMeter/'
    archivedDestination= root+eachlab+'//Import//'+inst+'//'
#     if(mode=='T'):
#         importDest= root+eachlab+'//impf//testing//'
#     else:
#         importDest= root+eachlab+'//impf//'
#     if os.path.exists(path):
#         pass
#     else:
#         os.makedirs(path)
#     if os.path.exists(importDest):
#         pass
#     else:
#         os.makedirs(importDest)
    files = os.listdir(path)
    for filename in files:
        if fnmatch.fnmatch(filename,'*.csv'):
            print (filename)
            datapath = path + str(filename)
            rawdf = pd.read_csv(datapath, sep=',', index_col=False)
            rawdf = rawdf.rename(index=str, columns={'�C': '°C'})
            rawdf = rawdf[pd.notnull(rawdf['Sample Number'])]
            rawdf = rawdf.rename(index=str, columns={'Sample Number': 'sample_number'})
            #rawdf = rawdf.astype({"sample_number": int})
            #rawdf['rep_count'] = rawdf['Sample Number'].apply(lambda x: str(x).split('-')[1])
            #rawdf['sample_number'] = rawdf['Sample Number'].apply(lambda x: str(x).split('-')[0])
            # LIMS will only read the last latest reading for each sample
            grouped_data = rawdf.groupby('sample_number')
            last_reading_per_sample = (pd.concat([grouped_data.tail(1)]).drop_duplicates().reset_index(drop=True))
            #print(last_reading_per_sample)
            # Creating new column (Entry) and set value as water Activity for (now)
            last_reading_per_sample['Entry'] = last_reading_per_sample["Water Activity"]
            last_reading_per_sample['Name'] = "Aw Reading"
            #print(last_reading_per_sample)
            # Duplicate the current 1 row into 2 rows
            d = last_reading_per_sample.loc[last_reading_per_sample.index.repeat(2),:].reset_index(drop=True)
            # keep="first" argument will mark 1st row as non-duplicate
            idx_duplicate = d.duplicated(keep="first")
            #print(d.columns)
            # Setting value of °C column in the second duplicated row in the Entry Column
            d.loc[idx_duplicate,"Entry"] = d['°C'].astype(object)
            d.loc[idx_duplicate,"Name"] = 'Aw Temp'
            # Removing 'Water Activity' & '°C' column as they are now copied inside Entry column
            column_remove=['Water Activity','°C']
            df = d[list(set(d.columns)-set(column_remove))]
            
            sampleNumbersList = list(df['sample_number'].unique())
            print(sampleNumbersList)
            
            # Getting sample list from db
            databasedf = getDataFromDB(sampleNumbersList)
            databasedf['Name'] = databasedf['Name'].map({'Water Activity': 'Aw Reading', 'Temperature': 'Aw Temp'})
            df = df.astype({"sample_number": int})
            #print(df)
            #print('Databasedf------- :',databasedf)
            dfResult = df.merge(databasedf, how='inner', on=['sample_number','Replicate','Name'])
            #print('dfResult-----> ',dfResult)
            #Needs to be updated according to Database columns
            dfResult = dfResult.rename(index=str,
                                       columns={'sample_number': 'Sample.Sample_Number',
                                                'Entry': 'Result.Entry',
                                                'test_number': 'Test.Test_Number',
                                                'result_number':'Result.Result_Number',
                                                'Replicate':'Test.Replicate_Count'})
           
            dfResult['Result.Entered_By'] = 'WATERACTIVITY'
            dfResultFinal = dfResult.sort_values(by=['Sample.Sample_Number'])
            dfResultFinal = dfResultFinal[colsOrder]
            #print(dfResultFinal.to_markdown())
           # writeFile(dfResultFinal, eachlab, inst, path, importDest, datapath, filename)
print("Process Done!")